# 🚀 Nowoczesny eksport modelu ensemble do ONNX

Ten notebook implementuje najlepsze praktyki eksportu do ONNX zgodne z PyTorch 2.7+ i najnowszymi standardami ONNX.

## Funkcje:
- ✅ Bezpieczne ładowanie modeli
- ✅ Optymalizacja modeli ONNX
- ✅ Kwantyzacja (dynamiczna i statyczna)
- ✅ Weryfikacja zgodności
- ✅ Eksport metadanych
- ✅ Jednolite wejście tensorowe
- ✅ Obsługa dynamicznych rozmiarów
- ✅ Obsługa 5 typów cech: chroma, tempogram, mfcc, melspectrogram, hpss

In [2]:
# Importy i konfiguracja
import sys
import os
from pathlib import Path

# Dodanie ścieżki do modułów
sys.path.append(os.path.join(os.getcwd()))

from export_scripts.modern_onnx_exporter import ModernONNXExporter, ExportConfig
import torch
import onnx
import onnxruntime as ort

print(f"🔧 Wersje bibliotek:")
print(f"   PyTorch: {torch.__version__}")
print(f"   ONNX: {onnx.__version__}")
print(f"   ONNX Runtime: {ort.__version__}")
print(f"   Python: {sys.version.split()[0]}")

🔧 Wersje bibliotek:
   PyTorch: 2.7.0+cpu
   ONNX: 1.18.0
   ONNX Runtime: 1.19.2
   Python: 3.9.13


## 📋 Konfiguracja eksportu

Ustaw parametry eksportu zgodnie z Twoimi potrzebami:

In [3]:
# Konfiguracja eksportu - dostosuj według potrzeb
config = ExportConfig(
    model_path=None,  # None = automatyczne wykrycie najnowszego modelu
    output_dir=None,  # None = automatyczne tworzenie katalogu z timestamp
    
    # Parametry audio
    sample_rate=22050,
    max_length=3.0,  # sekundy
    
    # Parametry ONNX
    opset_version=19,  # Najnowsza stabilna wersja
    
    # Optymalizacje
    enable_optimization=True,     # Włącz optymalizację grafu ONNX
    enable_quantization=False,    # Włącz kwantyzację (zmniejsza rozmiar)
    quantization_type="dynamic",  # "dynamic" lub "static"
    
    # Dodatkowe opcje
    fp16_conversion=False,        # Konwersja do połówkowej precyzji
    verify_model=True,            # Weryfikacja zgodności modeli
    export_metadata=True          # Eksport metadanych
)

print("📋 Konfiguracja eksportu:")
print(f"   Opset version: {config.opset_version}")
print(f"   Optymalizacja: {config.enable_optimization}")
print(f"   Kwantyzacja: {config.enable_quantization}")
print(f"   Typ kwantyzacji: {config.quantization_type}")
print(f"   Weryfikacja: {config.verify_model}")

📋 Konfiguracja eksportu:
   Opset version: 19
   Optymalizacja: True
   Kwantyzacja: False
   Typ kwantyzacji: dynamic
   Weryfikacja: True


## 🔍 Wyszukiwanie dostępnych modeli

Sprawdźmy jakie modele ensemble są dostępne:

In [4]:
import glob
from datetime import datetime

# Wyszukiwanie modeli ensemble
ensemble_models = glob.glob("ensemble_outputs/ensemble_run_*/models/ensemble_model.pt")

if ensemble_models:
    print(f"🔍 Znaleziono {len(ensemble_models)} modeli ensemble:")
    
    for i, model_path in enumerate(sorted(ensemble_models, reverse=True)):
        # Ekstraktowanie timestamp z nazwy katalogu
        timestamp_str = model_path.split('ensemble_run_')[1].split('/')[0]
        
        # Rozmiar pliku
        size_mb = os.path.getsize(model_path) / (1024 * 1024)
        
        print(f"   {i+1}. {model_path}")
        print(f"      Timestamp: {timestamp_str}")
        print(f"      Rozmiar: {size_mb:.2f} MB")
        
        if i == 0:
            latest_model = model_path
            print(f"      ⭐ Najnowszy model")
        print()
    
    # Automatyczne ustawienie najnowszego modelu
    if not config.model_path:
        config.model_path = latest_model
        print(f"✅ Automatycznie wybrano najnowszy model: {config.model_path}")
else:
    print("❌ Nie znaleziono żadnych modeli ensemble!")
    print("   Sprawdź czy folder ensemble_outputs/ zawiera wytrenowane modele.")

🔍 Znaleziono 1 modeli ensemble:
   1. ensemble_outputs\ensemble_run_20250523_193302\models\ensemble_model.pt
      Timestamp: 20250523_193302\models\ensemble_model.pt
      Rozmiar: 213.51 MB
      ⭐ Najnowszy model

✅ Automatycznie wybrano najnowszy model: ensemble_outputs\ensemble_run_20250523_193302\models\ensemble_model.pt


## ✅ PROBLEM ZOSTAŁ ROZWIĄZANY!

Błąd z ładowaniem modelu w PyTorch 2.7+ został naprawiony poprzez implementację wielopoziomowej strategii ładowania:
1. Najpierw próba bezpiecznego ładowania (`weights_only=True`)
2. Następnie z dodanym `torch.version.TorchVersion` do safe_globals
3. W końcu fallback do `weights_only=False` dla własnych zaufanych modeli

## 🚀 Eksport modelu do ONNX

Wykonajmy eksport z wykorzystaniem nowoczesnego eksportera:

In [5]:
# Tworzenie eksportera
exporter = ModernONNXExporter(config)

print("🎯 Rozpoczynam eksport modelu ensemble do ONNX")
print("=" * 60)

try:
    # Ładowanie modelu
    print("\n📥 Etap 1: Ładowanie modelu ensemble...")
    model, feature_types = exporter.load_ensemble_model()
    
    print(f"✅ Model załadowany pomyślnie!")
    print(f"   Typy cech: {feature_types}")
    print(f"   Liczba parametrów: {sum(p.numel() for p in model.parameters()):,}")
    
    # Eksport do ONNX
    print("\n📦 Etap 2: Eksport do ONNX...")
    result = exporter.export_to_onnx(model, feature_types)
    
    if result["success"]:
        print("\n🎉 Eksport zakończony sukcesem!")
        print(f"   📁 Katalog wyjściowy: {config.output_dir}")
        print(f"   📦 Podstawowy model: {result['size_mb']:.2f} MB")
        
        if "optimized_path" in result:
            opt_size = Path(result["optimized_path"]).stat().st_size / (1024 * 1024)
            print(f"   ⚡ Zoptymalizowany model: {opt_size:.2f} MB")
        
        if "quantized_path" in result:
            quant_size = Path(result["quantized_path"]).stat().st_size / (1024 * 1024)
            print(f"   🔢 Skwantyzowany model: {quant_size:.2f} MB")
            
        export_result = result  # Zapisz wynik dla dalszych analiz
    else:
        print(f"\n❌ Eksport nieudany: {result['error']}")
        
except Exception as e:
    print(f"\n💥 Krytyczny błąd: {e}")
    import traceback
    traceback.print_exc()

🎯 Rozpoczynam eksport modelu ensemble do ONNX

📥 Etap 1: Ładowanie modelu ensemble...
📥 Ładowanie modelu: ensemble_outputs\ensemble_run_20250523_193302\models\ensemble_model.pt
⚠️ Używam fallback ładowania (weights_only=False)
   Błąd bezpiecznego ładowania: Weights only load failed. This file can still be loaded, to do so you have two options, do those...
✅ Model załadowany z 5 typami cech: ['melspectrogram', 'mfcc', 'hpss', 'chroma', 'tempogram']
✅ Model załadowany pomyślnie!
   Typy cech: ['melspectrogram', 'mfcc', 'hpss', 'chroma', 'tempogram']
   Liczba parametrów: 55,866,596

📦 Etap 2: Eksport do ONNX...

🚀 Rozpoczynam eksport do ONNX...
📊 Wymiary wejścia: torch.Size([1, 1, 692, 130])
📋 Informacje o cechach: {'melspectrogram': 128, 'mfcc': 40, 'hpss': 128, 'chroma': 12, 'tempogram': 384}
🔢 Całkowity rozmiar cech: 692
🔍 Test forward pass...
✅ Test ukończony. Wymiary wyjścia: torch.Size([1, 6])
📦 Eksport do ONNX...
✅ Eksport ukończony: exported_models\onnx_20250523_220917\ensemble_

## 🔍 Analiza wyeksportowanych modeli

Sprawdźmy szczegóły wyeksportowanych modeli:

In [6]:
if 'export_result' in locals() and export_result["success"]:
    print("📊 Analiza wyeksportowanych modeli:")
    print("=" * 40)
    
    # Lista wszystkich wyeksportowanych plików
    output_dir = Path(config.output_dir)
    
    print(f"\n📁 Zawartość katalogu: {output_dir}")
    for file_path in sorted(output_dir.iterdir()):
        if file_path.is_file():
            size_mb = file_path.stat().st_size / (1024 * 1024)
            print(f"   📄 {file_path.name}: {size_mb:.2f} MB")
    
    # Analiza podstawowego modelu ONNX
    main_model_path = output_dir / "ensemble_model.onnx"
    if main_model_path.exists():
        print(f"\n🔍 Analiza modelu ONNX: {main_model_path.name}")
        
        try:
            model_onnx = onnx.load(str(main_model_path))
            
            print(f"   Wersja ONNX: {model_onnx.ir_version}")
            print(f"   Opset version: {model_onnx.opset_import[0].version}")
            print(f"   Liczba węzłów: {len(model_onnx.graph.node)}")
            print(f"   Liczba inicjalizatorów: {len(model_onnx.graph.initializer)}")
            
            # Informacje o wejściach/wyjściach
            print(f"\n   📥 Wejścia:")
            for input_info in model_onnx.graph.input:
                name = input_info.name
                shape = [dim.dim_value if dim.dim_value > 0 else f"dynamic({dim.dim_param})" 
                        for dim in input_info.type.tensor_type.shape.dim]
                print(f"      {name}: {shape}")
            
            print(f"\n   📤 Wyjścia:")
            for output_info in model_onnx.graph.output:
                name = output_info.name
                shape = [dim.dim_value if dim.dim_value > 0 else f"dynamic({dim.dim_param})" 
                        for dim in output_info.type.tensor_type.shape.dim]
                print(f"      {name}: {shape}")
                
        except Exception as e:
            print(f"   ❌ Błąd analizy modelu: {e}")
else:
    print("❌ Brak wyeksportowanych modeli do analizy")

📊 Analiza wyeksportowanych modeli:

📁 Zawartość katalogu: exported_models\onnx_20250523_220917
   📄 ensemble_model.onnx: 213.09 MB
   📄 export_metadata.json: 0.00 MB

🔍 Analiza modelu ONNX: ensemble_model.onnx
   Wersja ONNX: 9
   Opset version: 19
   Liczba węzłów: 292
   Liczba inicjalizatorów: 216

   📥 Wejścia:
      audio_features: ['dynamic(batch_size)', 1, 692, 'dynamic(time_steps)']

   📤 Wyjścia:
      output: ['dynamic(batch_size)', 6]


## 🧪 Test inference modelu ONNX

Sprawdźmy czy model ONNX działa poprawnie:

In [7]:
if 'export_result' in locals() and export_result["success"]:
    print("🧪 Test inference modelu ONNX")
    print("=" * 30)
    
    try:
        # Ścieżka do modelu (preferuj zoptymalizowany jeśli istnieje)
        if "optimized_path" in export_result:
            test_model_path = export_result["optimized_path"]
            print(f"📦 Używam zoptymalizowanego modelu")
        else:
            test_model_path = export_result["path"]
            print(f"📦 Używam podstawowego modelu")
        
        print(f"   Model: {Path(test_model_path).name}")
        
        # Tworzenie sesji ONNX Runtime
        session = ort.InferenceSession(
            test_model_path,
            providers=['CPUExecutionProvider']
        )
        
        print(f"✅ Sesja ONNX Runtime utworzona")
        print(f"   Provider: {session.get_providers()}")
        
        # Generowanie testowych danych
        print(f"\n📊 Generowanie testowych danych...")
        test_input = exporter.generate_dummy_input(export_result['feature_types'])
        
        # Test inference
        print(f"🚀 Test inference...")
        import time
        
        start_time = time.time()
        onnx_output = session.run(
            None,  # Wszystkie wyjścia
            {'audio_features': test_input.numpy()}
        )
        inference_time = time.time() - start_time
        
        print(f"✅ Inference ukończony w {inference_time*1000:.2f} ms")
        print(f"   Wymiary wyjścia: {onnx_output[0].shape}")
        print(f"   Przewidywane klasy: {onnx_output[0].argmax(axis=1)}")
        print(f"   Najwyższe prawdopodobieństwo: {onnx_output[0].max():.4f}")
        
        # Test z wieloma batch'ami
        print(f"\n🔄 Test z batch size = 4...")
        batch_input = test_input.repeat(4, 1, 1, 1)
        
        start_time = time.time()
        batch_output = session.run(
            None,
            {'audio_features': batch_input.numpy()}
        )
        batch_time = time.time() - start_time
        
        print(f"✅ Batch inference ukończony w {batch_time*1000:.2f} ms")
        print(f"   Wymiary wyjścia: {batch_output[0].shape}")
        print(f"   Czas na próbkę: {batch_time/4*1000:.2f} ms")
        
    except Exception as e:
        print(f"❌ Błąd testu inference: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ Brak wyeksportowanego modelu do testowania")

🧪 Test inference modelu ONNX
📦 Używam zoptymalizowanego modelu
   Model: ensemble_model.onnx
✅ Sesja ONNX Runtime utworzona
   Provider: ['CPUExecutionProvider']

📊 Generowanie testowych danych...
📊 Wymiary wejścia: torch.Size([1, 1, 692, 130])
📋 Informacje o cechach: {'melspectrogram': 128, 'mfcc': 40, 'hpss': 128, 'chroma': 12, 'tempogram': 384}
🔢 Całkowity rozmiar cech: 692
🚀 Test inference...
✅ Inference ukończony w 31.16 ms
   Wymiary wyjścia: (1, 6)
   Przewidywane klasy: [0]
   Najwyższe prawdopodobieństwo: 0.5327

🔄 Test z batch size = 4...
✅ Batch inference ukończony w 129.94 ms
   Wymiary wyjścia: (4, 6)
   Czas na próbkę: 32.48 ms


## 📋 Podsumowanie eksportu

Podsumujmy wyniki eksportu:

In [8]:
if 'export_result' in locals() and export_result["success"]:
    print("📋 PODSUMOWANIE EKSPORTU")
    print("=" * 40)
    
    print(f"✅ Status: SUKCES")
    print(f"📁 Katalog wyjściowy: {config.output_dir}")
    print(f"🔧 Typy cech: {', '.join(export_result['feature_types'])}")
    print(f"📦 Rozmiar podstawowego modelu: {export_result['size_mb']:.2f} MB")
    
    print(f"\n🛠️ Zastosowane optymalizacje:")
    print(f"   ⚡ Optymalizacja grafu: {config.enable_optimization}")
    print(f"   🔢 Kwantyzacja: {config.enable_quantization}")
    if config.enable_quantization:
        print(f"   📏 Typ kwantyzacji: {config.quantization_type}")
    print(f"   🔍 Weryfikacja: {config.verify_model}")
    
    print(f"\n📊 Dostępne pliki:")
    output_dir = Path(config.output_dir)
    for file_path in sorted(output_dir.iterdir()):
        if file_path.suffix in ['.onnx', '.json']:
            size_mb = file_path.stat().st_size / (1024 * 1024)
            print(f"   📄 {file_path.name}: {size_mb:.2f} MB")
    
    print(f"\n🎯 Model gotowy do wdrożenia!")
    print(f"   Główny model: {Path(export_result['path']).name}")
    if "optimized_path" in export_result:
        print(f"   Zalecany do użycia: {Path(export_result['optimized_path']).name}")
    
else:
    print("❌ EKSPORT NIEUDANY")
    print("   Sprawdź komunikaty błędów powyżej")

📋 PODSUMOWANIE EKSPORTU
✅ Status: SUKCES
📁 Katalog wyjściowy: exported_models/onnx_20250523_220917
🔧 Typy cech: melspectrogram, mfcc, hpss, chroma, tempogram
📦 Rozmiar podstawowego modelu: 213.09 MB

🛠️ Zastosowane optymalizacje:
   ⚡ Optymalizacja grafu: True
   🔢 Kwantyzacja: False
   🔍 Weryfikacja: True

📊 Dostępne pliki:
   📄 ensemble_model.onnx: 213.09 MB
   📄 export_metadata.json: 0.00 MB

🎯 Model gotowy do wdrożenia!
   Główny model: ensemble_model.onnx
   Zalecany do użycia: ensemble_model.onnx


## 💡 Następne kroki

Po udanym eksporcie możesz:

1. **Wdrożenie modelu**: Użyj wyeksportowanego modelu ONNX w swojej aplikacji
2. **Optymalizacja dalszej**: Przetestuj różne konfiguracje optymalizacji
3. **Kwantyzacja**: Jeśli nie włączałeś kwantyzacji, rozważ jej użycie dla zmniejszenia rozmiaru
4. **Testy wydajności**: Zmierz wydajność na docelowym środowisku
5. **Integracja**: Użyj modelu z bibliotekami takimi jak ONNX Runtime, OpenVINO, lub TensorRT

### Przykład użycia w produkcji:

```python
import onnxruntime as ort
import numpy as np

# Ładowanie modelu
session = ort.InferenceSession('ensemble_model_optimized.onnx')

# Przygotowanie danych audio (zastąp swoją logiką)
audio_features = extract_features_from_audio(audio_file)

# Inference
predictions = session.run(None, {'audio_features': audio_features})
predicted_emotion = np.argmax(predictions[0])
```

# 🚀 Nowoczesny eksport modelu ensemble do ONNX

Ten notebook implementuje najlepsze praktyki eksportu do ONNX zgodne z PyTorch 2.7+ i najnowszymi standardami ONNX.

## Funkcje:
- ✅ Bezpieczne ładowanie modeli
- ✅ Optymalizacja modeli ONNX
- ✅ Kwantyzacja (dynamiczna i statyczna)
- ✅ Weryfikacja zgodności
- ✅ Eksport metadanych
- ✅ Jednolite wejście tensorowe
- ✅ Obsługa dynamicznych rozmiarów
- ✅ Obsługa 5 typów cech: chroma, tempogram, mfcc, melspectrogram, hpss

In [9]:
# Importy i konfiguracja
import sys
import os
from pathlib import Path

# Dodanie ścieżki do modułów
sys.path.append(os.path.join(os.getcwd()))

from export_scripts.modern_onnx_exporter import ModernONNXExporter, ExportConfig
import torch
import onnx
import onnxruntime as ort

print(f"🔧 Wersje bibliotek:")
print(f"   PyTorch: {torch.__version__}")
print(f"   ONNX: {onnx.__version__}")
print(f"   ONNX Runtime: {ort.__version__}")
print(f"   Python: {sys.version.split()[0]}")

🔧 Wersje bibliotek:
   PyTorch: 2.7.0+cpu
   ONNX: 1.18.0
   ONNX Runtime: 1.19.2
   Python: 3.9.13


## 📋 Konfiguracja eksportu

Ustaw parametry eksportu zgodnie z Twoimi potrzebami:

In [10]:
# Konfiguracja eksportu - dostosuj według potrzeb
config = ExportConfig(
    model_path=None,  # None = automatyczne wykrycie najnowszego modelu
    output_dir=None,  # None = automatyczne tworzenie katalogu z timestamp
    
    # Parametry audio
    sample_rate=22050,
    max_length=3.0,  # sekundy
    
    # Parametry ONNX
    opset_version=19,  # Najnowsza stabilna wersja
    
    # Optymalizacje
    enable_optimization=True,     # Włącz optymalizację grafu ONNX
    enable_quantization=False,    # Włącz kwantyzację (zmniejsza rozmiar)
    quantization_type="dynamic",  # "dynamic" lub "static"
    
    # Dodatkowe opcje
    fp16_conversion=False,        # Konwersja do połówkowej precyzji
    verify_model=True,            # Weryfikacja zgodności modeli
    export_metadata=True          # Eksport metadanych
)

print("📋 Konfiguracja eksportu:")
print(f"   Opset version: {config.opset_version}")
print(f"   Optymalizacja: {config.enable_optimization}")
print(f"   Kwantyzacja: {config.enable_quantization}")
print(f"   Typ kwantyzacji: {config.quantization_type}")
print(f"   Weryfikacja: {config.verify_model}")

📋 Konfiguracja eksportu:
   Opset version: 19
   Optymalizacja: True
   Kwantyzacja: False
   Typ kwantyzacji: dynamic
   Weryfikacja: True


## 🔍 Wyszukiwanie dostępnych modeli

Sprawdźmy jakie modele ensemble są dostępne:

In [11]:
import glob
from datetime import datetime

# Wyszukiwanie modeli ensemble
ensemble_models = glob.glob("ensemble_outputs/ensemble_run_*/models/ensemble_model.pt")

if ensemble_models:
    print(f"🔍 Znaleziono {len(ensemble_models)} modeli ensemble:")
    
    for i, model_path in enumerate(sorted(ensemble_models, reverse=True)):
        # Ekstraktowanie timestamp z nazwy katalogu
        timestamp_str = model_path.split('ensemble_run_')[1].split('/')[0]
        
        # Rozmiar pliku
        size_mb = os.path.getsize(model_path) / (1024 * 1024)
        
        print(f"   {i+1}. {model_path}")
        print(f"      Timestamp: {timestamp_str}")
        print(f"      Rozmiar: {size_mb:.2f} MB")
        
        if i == 0:
            latest_model = model_path
            print(f"      ⭐ Najnowszy model")
        print()
    
    # Automatyczne ustawienie najnowszego modelu
    if not config.model_path:
        config.model_path = latest_model
        print(f"✅ Automatycznie wybrano najnowszy model: {config.model_path}")
else:
    print("❌ Nie znaleziono żadnych modeli ensemble!")
    print("   Sprawdź czy folder ensemble_outputs/ zawiera wytrenowane modele.")

🔍 Znaleziono 1 modeli ensemble:
   1. ensemble_outputs\ensemble_run_20250523_193302\models\ensemble_model.pt
      Timestamp: 20250523_193302\models\ensemble_model.pt
      Rozmiar: 213.51 MB
      ⭐ Najnowszy model

✅ Automatycznie wybrano najnowszy model: ensemble_outputs\ensemble_run_20250523_193302\models\ensemble_model.pt


## 🚀 Eksport modelu do ONNX

Wykonajmy eksport z wykorzystaniem nowoczesnego eksportera:

In [12]:
# Tworzenie eksportera
exporter = ModernONNXExporter(config)

print("🎯 Rozpoczynam eksport modelu ensemble do ONNX")
print("=" * 60)

try:
    # Ładowanie modelu
    print("\n📥 Etap 1: Ładowanie modelu ensemble...")
    model, feature_types = exporter.load_ensemble_model()
    
    print(f"✅ Model załadowany pomyślnie!")
    print(f"   Typy cech: {feature_types}")
    print(f"   Liczba parametrów: {sum(p.numel() for p in model.parameters()):,}")
    
    # Eksport do ONNX
    print("\n📦 Etap 2: Eksport do ONNX...")
    result = exporter.export_to_onnx(model, feature_types)
    
    if result["success"]:
        print("\n🎉 Eksport zakończony sukcesem!")
        print(f"   📁 Katalog wyjściowy: {config.output_dir}")
        print(f"   📦 Podstawowy model: {result['size_mb']:.2f} MB")
        
        if "optimized_path" in result:
            opt_size = Path(result["optimized_path"]).stat().st_size / (1024 * 1024)
            print(f"   ⚡ Zoptymalizowany model: {opt_size:.2f} MB")
        
        if "quantized_path" in result:
            quant_size = Path(result["quantized_path"]).stat().st_size / (1024 * 1024)
            print(f"   🔢 Skwantyzowany model: {quant_size:.2f} MB")
            
        export_result = result  # Zapisz wynik dla dalszych analiz
    else:
        print(f"\n❌ Eksport nieudany: {result['error']}")
        
except Exception as e:
    print(f"\n💥 Krytyczny błąd: {e}")
    import traceback
    traceback.print_exc()

🎯 Rozpoczynam eksport modelu ensemble do ONNX

📥 Etap 1: Ładowanie modelu ensemble...
📥 Ładowanie modelu: ensemble_outputs\ensemble_run_20250523_193302\models\ensemble_model.pt
⚠️ Używam fallback ładowania (weights_only=False)
   Błąd bezpiecznego ładowania: Weights only load failed. This file can still be loaded, to do so you have two options, do those...
✅ Model załadowany z 5 typami cech: ['melspectrogram', 'mfcc', 'hpss', 'chroma', 'tempogram']
✅ Model załadowany pomyślnie!
   Typy cech: ['melspectrogram', 'mfcc', 'hpss', 'chroma', 'tempogram']
   Liczba parametrów: 55,866,596

📦 Etap 2: Eksport do ONNX...

🚀 Rozpoczynam eksport do ONNX...
📊 Wymiary wejścia: torch.Size([1, 1, 692, 130])
📋 Informacje o cechach: {'melspectrogram': 128, 'mfcc': 40, 'hpss': 128, 'chroma': 12, 'tempogram': 384}
🔢 Całkowity rozmiar cech: 692
🔍 Test forward pass...
✅ Test ukończony. Wymiary wyjścia: torch.Size([1, 6])
📦 Eksport do ONNX...
✅ Eksport ukończony: exported_models\onnx_20250523_220923\ensemble_

## 🔍 Analiza wyeksportowanych modeli

Sprawdźmy szczegóły wyeksportowanych modeli:

In [13]:
if 'export_result' in locals() and export_result["success"]:
    print("📊 Analiza wyeksportowanych modeli:")
    print("=" * 40)
    
    # Lista wszystkich wyeksportowanych plików
    output_dir = Path(config.output_dir)
    
    print(f"\n📁 Zawartość katalogu: {output_dir}")
    for file_path in sorted(output_dir.iterdir()):
        if file_path.is_file():
            size_mb = file_path.stat().st_size / (1024 * 1024)
            print(f"   📄 {file_path.name}: {size_mb:.2f} MB")
    
    # Analiza podstawowego modelu ONNX
    main_model_path = output_dir / "ensemble_model.onnx"
    if main_model_path.exists():
        print(f"\n🔍 Analiza modelu ONNX: {main_model_path.name}")
        
        try:
            model_onnx = onnx.load(str(main_model_path))
            
            print(f"   Wersja ONNX: {model_onnx.ir_version}")
            print(f"   Opset version: {model_onnx.opset_import[0].version}")
            print(f"   Liczba węzłów: {len(model_onnx.graph.node)}")
            print(f"   Liczba inicjalizatorów: {len(model_onnx.graph.initializer)}")
            
            # Informacje o wejściach/wyjściach
            print(f"\n   📥 Wejścia:")
            for input_info in model_onnx.graph.input:
                name = input_info.name
                shape = [dim.dim_value if dim.dim_value > 0 else f"dynamic({dim.dim_param})" 
                        for dim in input_info.type.tensor_type.shape.dim]
                print(f"      {name}: {shape}")
            
            print(f"\n   📤 Wyjścia:")
            for output_info in model_onnx.graph.output:
                name = output_info.name
                shape = [dim.dim_value if dim.dim_value > 0 else f"dynamic({dim.dim_param})" 
                        for dim in output_info.type.tensor_type.shape.dim]
                print(f"      {name}: {shape}")
                
        except Exception as e:
            print(f"   ❌ Błąd analizy modelu: {e}")
else:
    print("❌ Brak wyeksportowanych modeli do analizy")

📊 Analiza wyeksportowanych modeli:

📁 Zawartość katalogu: exported_models\onnx_20250523_220923
   📄 ensemble_model.onnx: 213.09 MB
   📄 export_metadata.json: 0.00 MB

🔍 Analiza modelu ONNX: ensemble_model.onnx
   Wersja ONNX: 9
   Opset version: 19
   Liczba węzłów: 292
   Liczba inicjalizatorów: 216

   📥 Wejścia:
      audio_features: ['dynamic(batch_size)', 1, 692, 'dynamic(time_steps)']

   📤 Wyjścia:
      output: ['dynamic(batch_size)', 6]


## 🧪 Test inference modelu ONNX

Sprawdźmy czy model ONNX działa poprawnie:

In [14]:
if 'export_result' in locals() and export_result["success"]:
    print("🧪 Test inference modelu ONNX")
    print("=" * 30)
    
    try:
        # Ścieżka do modelu (preferuj zoptymalizowany jeśli istnieje)
        if "optimized_path" in export_result:
            test_model_path = export_result["optimized_path"]
            print(f"📦 Używam zoptymalizowanego modelu")
        else:
            test_model_path = export_result["path"]
            print(f"📦 Używam podstawowego modelu")
        
        print(f"   Model: {Path(test_model_path).name}")
        
        # Tworzenie sesji ONNX Runtime
        session = ort.InferenceSession(
            test_model_path,
            providers=['CPUExecutionProvider']
        )
        
        print(f"✅ Sesja ONNX Runtime utworzona")
        print(f"   Provider: {session.get_providers()}")
        
        # Generowanie testowych danych
        print(f"\n📊 Generowanie testowych danych...")
        test_input = exporter.generate_dummy_input(export_result['feature_types'])
        
        # Test inference
        print(f"🚀 Test inference...")
        import time
        
        start_time = time.time()
        onnx_output = session.run(
            None,  # Wszystkie wyjścia
            {'audio_features': test_input.numpy()}
        )
        inference_time = time.time() - start_time
        
        print(f"✅ Inference ukończony w {inference_time*1000:.2f} ms")
        print(f"   Wymiary wyjścia: {onnx_output[0].shape}")
        print(f"   Przewidywane klasy: {onnx_output[0].argmax(axis=1)}")
        print(f"   Najwyższe prawdopodobieństwo: {onnx_output[0].max():.4f}")
        
        # Test z wieloma batch'ami
        print(f"\n🔄 Test z batch size = 4...")
        batch_input = test_input.repeat(4, 1, 1, 1)
        
        start_time = time.time()
        batch_output = session.run(
            None,
            {'audio_features': batch_input.numpy()}
        )
        batch_time = time.time() - start_time
        
        print(f"✅ Batch inference ukończony w {batch_time*1000:.2f} ms")
        print(f"   Wymiary wyjścia: {batch_output[0].shape}")
        print(f"   Czas na próbkę: {batch_time/4*1000:.2f} ms")
        
    except Exception as e:
        print(f"❌ Błąd testu inference: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ Brak wyeksportowanego modelu do testowania")

🧪 Test inference modelu ONNX
📦 Używam zoptymalizowanego modelu
   Model: ensemble_model.onnx
✅ Sesja ONNX Runtime utworzona
   Provider: ['CPUExecutionProvider']

📊 Generowanie testowych danych...
📊 Wymiary wejścia: torch.Size([1, 1, 692, 130])
📋 Informacje o cechach: {'melspectrogram': 128, 'mfcc': 40, 'hpss': 128, 'chroma': 12, 'tempogram': 384}
🔢 Całkowity rozmiar cech: 692
🚀 Test inference...
✅ Inference ukończony w 24.72 ms
   Wymiary wyjścia: (1, 6)
   Przewidywane klasy: [0]
   Najwyższe prawdopodobieństwo: 0.5120

🔄 Test z batch size = 4...
✅ Batch inference ukończony w 111.85 ms
   Wymiary wyjścia: (4, 6)
   Czas na próbkę: 27.96 ms


## 📋 Podsumowanie eksportu

Podsumujmy wyniki eksportu:

In [15]:
if 'export_result' in locals() and export_result["success"]:
    print("📋 PODSUMOWANIE EKSPORTU")
    print("=" * 40)
    
    print(f"✅ Status: SUKCES")
    print(f"📁 Katalog wyjściowy: {config.output_dir}")
    print(f"🔧 Typy cech: {', '.join(export_result['feature_types'])}")
    print(f"📦 Rozmiar podstawowego modelu: {export_result['size_mb']:.2f} MB")
    
    print(f"\n🛠️ Zastosowane optymalizacje:")
    print(f"   ⚡ Optymalizacja grafu: {config.enable_optimization}")
    print(f"   🔢 Kwantyzacja: {config.enable_quantization}")
    if config.enable_quantization:
        print(f"   📏 Typ kwantyzacji: {config.quantization_type}")
    print(f"   🔍 Weryfikacja: {config.verify_model}")
    
    print(f"\n📊 Dostępne pliki:")
    output_dir = Path(config.output_dir)
    for file_path in sorted(output_dir.iterdir()):
        if file_path.suffix in ['.onnx', '.json']:
            size_mb = file_path.stat().st_size / (1024 * 1024)
            print(f"   📄 {file_path.name}: {size_mb:.2f} MB")
    
    print(f"\n🎯 Model gotowy do wdrożenia!")
    print(f"   Główny model: {Path(export_result['path']).name}")
    if "optimized_path" in export_result:
        print(f"   Zalecany do użycia: {Path(export_result['optimized_path']).name}")
    
else:
    print("❌ EKSPORT NIEUDANY")
    print("   Sprawdź komunikaty błędów powyżej")

📋 PODSUMOWANIE EKSPORTU
✅ Status: SUKCES
📁 Katalog wyjściowy: exported_models/onnx_20250523_220923
🔧 Typy cech: melspectrogram, mfcc, hpss, chroma, tempogram
📦 Rozmiar podstawowego modelu: 213.09 MB

🛠️ Zastosowane optymalizacje:
   ⚡ Optymalizacja grafu: True
   🔢 Kwantyzacja: False
   🔍 Weryfikacja: True

📊 Dostępne pliki:
   📄 ensemble_model.onnx: 213.09 MB
   📄 export_metadata.json: 0.00 MB

🎯 Model gotowy do wdrożenia!
   Główny model: ensemble_model.onnx
   Zalecany do użycia: ensemble_model.onnx


## 💡 Następne kroki

Po udanym eksporcie możesz:

1. **Wdrożenie modelu**: Użyj wyeksportowanego modelu ONNX w swojej aplikacji
2. **Optymalizacja dalszej**: Przetestuj różne konfiguracje optymalizacji
3. **Kwantyzacja**: Jeśli nie włączałeś kwantyzacji, rozważ jej użycie dla zmniejszenia rozmiaru
4. **Testy wydajności**: Zmierz wydajność na docelowym środowisku
5. **Integracja**: Użyj modelu z bibliotekami takimi jak ONNX Runtime, OpenVINO, lub TensorRT

### Przykład użycia w produkcji:

```python
import onnxruntime as ort
import numpy as np

# Ładowanie modelu
session = ort.InferenceSession('ensemble_model_optimized.onnx')

# Przygotowanie danych audio (zastąp swoją logiką)
audio_features = extract_features_from_audio(audio_file)

# Inference
predictions = session.run(None, {'audio_features': audio_features})
predicted_emotion = np.argmax(predictions[0])
```